Computer lab with Jupyter notebooks where the participants can investigate the firing properties of Izhikevich models with different firing patterns and try to hand tune parameters to fit some specified recordings.